In [1]:
import pickle
import re
from functions import *
import os
import itertools
import geopandas as gpd
import networkx as nx
from sumo import gen_net
from shapely.ops import cascaded_union
import osmnx.projection as projection
from pathlib import Path
import string
from copy import deepcopy

In [2]:
with open("../data/salt-lake-city/salt-lake-city.pkl", "rb") as f:
    data = pickle.load(f)

In [13]:
with open("../data/salt-lake-city/slc_new.pkl", "rb") as file:
    edges = pickle.load(file)

    road_info_dict = {}
for u, v, a in edges:
    if type(a["osmid"])==list:
        for ID in a["osmid"]:
            road_info_dict[ID] = a
    else:
        road_info_dict[a["osmid"]] = a

In [14]:
with open("../data/salt-lake-city/intersections.pkl", "rb") as f:
    intersection_dict = pickle.load(f)

In [5]:
with open("../data/salt-lake-city/intersections.pkl", "rb") as f:
    intersection_dict = pickle.load(f)

LETTERS = string.ascii_uppercase

In [15]:
missing_speed = 0
new_intersection_dict = {}
for i, intersection in intersection_dict.items():
    new_intersection = {}
    for j, road in intersection.items():
        if type(j) != int:
            new_intersection[j] = road
            continue
        new_road = road
        road_id = road["road_id"][0] if type(road["road_id"])==list else road["road_id"]
        try:
            speed, grad = road_info_dict[road_id]["maxspeed"], road_info_dict[road_id]["grade"]
        except:
            speed, grad = 40, 0
            missing_speed += 1
        new_road["speed"] = int(speed[0]) if type(speed) == tuple else int(speed)
        new_road["gradient"] = float(grad[0]) if type(grad) == tuple else float(grad)
        new_intersection[j] = new_road
    new_intersection_dict[i] = new_intersection
intersection_dict = new_intersection_dict

In [16]:
def combine_middle(intersection):
    new_intersection = {}
    for index, road in intersection.items():
        if type(index) != int:
            continue
        if type(road["lane_data"])==list and road["lane_data"][1]!=0:
            road["lane_data"][1] = 0
            road["lane_data"][0] = road["lane_data"][0] + 1
            road["turn_data"] = ["left"] + road["turn_data"]
            road["turn_data"][1] = ";".join([turn for turn in road["turn_data"][1].split(";") if turn != "left"])
            road["left_turn_only"] = True
        else:
            road["left_turn_only"] = False
        new_intersection[index] = road
    return new_intersection

new_intersection_dict = dict()
for index, intersection in intersection_dict.items():
    if len(intersection)>=3:
        new_intersection = combine_middle(intersection)
        new_intersection_dict[index] = new_intersection
intersection_dict = new_intersection_dict

In [17]:
turn_stats = {"right": [90.36729010723205, 9.78688759183337],
              "slight_right": [136.359, 14.275363708151186], 
              "through": [179.70755329451188, 11.986190506968194], 
              "slight_left": [224.36030149677322, 15.710788019224378],
              "left": [269.24091111044555, 8.70355104033318]}


def find_turn(turn, theta, turn_data, index, intersection):
    index_pos = sum([turn in turns for turns in turn_data[:index]])
    index_neg = sum([turn in turns for turns in turn_data[index+1:]])
    def get_degree(a, b):
        return a-b if a>b else a-b+360
    mean, std = turn_stats[turn]
    connection = max(intersection.items(), key=lambda x: scipy.stats.distributions.norm.pdf(get_degree(x[1]["theta"], theta), mean, std))[0]
    target_road = intersection[connection]
    if type(target_road["lane_data"])==list:
        target_lane = target_road["lane_data"][2]
    elif target_road["direction"]=="out":
        target_lane = target_road["lane_data"]
    else:
        return None
        
    if "left" in turn:
        index = max(0, target_lane-1-index_neg)
        return (connection, index)
    if "right" in turn:
        index = min(index_pos, target_lane-1)
        return (connection, index)
    else:
        index = min(target_lane-1, index_pos)
        return (connection, index)
    
def get_connectivity(intersection):
    for i, road1 in intersection.items():
        turn_data = road1["turn_data"][::-1]
        theta = road1["theta"]
        connectivity = {}
        for index, turns in enumerate(turn_data):
            pos, neg = index, len(turn_data)-index-1
            connections = [find_turn(turn, theta, turn_data, index, intersection) for turn in turns.split(";")]
            connections = [connection for connection in connections if connection]
            connectivity[index] = connections
        intersection[i]["connections"] = connectivity
    return intersection
    
def gen_sumo_intersection(intersection, flag=False):
    intersection = get_connectivity(intersection)
    new_intersection = []
    for index in range(len(intersection)):
        new_road = {}
        road = intersection[index]
        if type(road["lane_data"])==list:
            new_road["num_incoming"] = road["lane_data"][0]+road["lane_data"][1]
            new_road["num_outgoing"] = road["lane_data"][2]
        else:
            if road["direction"]=="in":
                new_road["num_incoming"] = road["lane_data"]
            else:
                new_road["num_outgoing"] = road["lane_data"]
        if road["left_turn_only"]:
            new_road["left_turn_only_length"] = 50
        if "num_incoming" not in new_road:
            new_road["num_incoming"]=0
        if "num_outgoing" not in new_road:
            new_road["num_outgoing"]=0
        new_road["length"] = road["length"]
        new_road["speed"] = road["speed"]
        if flag:
            new_road["connections"] = {}
        else:
            new_road["connections"] = {index:[(x, y) for x, y in lst] for index, lst in road["connections"].items()}
        new_road["slope"] = road["gradient"]
        new_intersection.append(new_road)
    return new_intersection

def build_intersection(intersection):
    def copy_road(road, idx):
        road = deepcopy(road)       
        if road["left_turn_only"]:
            road["lane_data"][0] = road["lane_data"][0] - 1
            road["turn_data"] = road["turn_data"][1:]
            road["left_turn_only"] = False
        road["lane_data"] = road["lane_data"][::-1]
        road["turn_data"] = get_default_turn(road["lane_data"][0])
        road["theta"] -= 90 * idx
        return road
    central = gen_sumo_intersection(deepcopy(intersection))
    neighbors = {}
    for index, road in intersection.items():
        road = deepcopy(road)
        flag = False
        if type(road["lane_data"])!=list:
            flag = True
            road["lane_data"] = [road["lane_data"], 0, 0] if road["direction"]=="in" else [0, 0, road["lane_data"]]
        new_intersection = {i: copy_road(road, i) for i in range(4)}
        neighbors[LETTERS[index]] = gen_sumo_intersection(new_intersection, flag)
        neighbors[LETTERS[index]][0]["num_incoming"], neighbors[LETTERS[index]][0]["num_outgoing"] = neighbors[LETTERS[index]][0]["num_outgoing"], neighbors[LETTERS[index]][0]["num_incoming"]
    return {"central": central, "neighbors": neighbors}

In [10]:
set(intersection_dict) - set([int(file.split(".")[0]) for file in os.listdir("../data/salt-lake-city/sumo/net/") if file.endswith("xml")])

set()

In [19]:
intersection

{0: {'length': 295.0187150421209,
  'geometry': [[40.7402338, -112.0248634],
   [40.7402244, -112.0246263],
   [40.7401819, -112.0235598],
   [40.7401773, -112.022215]],
  'road_id': 10137605,
  'theta': 357.0162298928654,
  'theta_long': 358.3933814664556,
  'direction': 'in',
  'lane_data': [1, 0, 2],
  'lane_data_source': 'osm',
  'turn_data': ['through;right'],
  'turn_data_source': 'default',
  'oneway': nan,
  'highway': 'secondary',
  'speed': 40,
  'gradient': 0.0024384000000000003,
  'left_turn_only': False},
 1: {'road_id': [743313660, 960215812],
  'length': 1134.8180171497056,
  'theta': 270.4496984841866,
  'theta_long': 270.0601589926163,
  'oneway': 'no',
  'direction': 'in',
  'lane_data': [2, 0, 2],
  'lane_data_source': 'osm/osm',
  'turn_data': ['left;through', 'through;right'],
  'turn_data_source': 'default',
  'highway': 'secondary',
  'geometry': [[[40.7402338, -112.0248634],
    [40.7400599, -112.0248657],
    [40.7399669, -112.0248647],
    [40.7324817, -112.02

In [18]:
from net import gen_net
# for index, intersection in intersection_dict.items():
for i in intersection_dict:
    print(i)
    intersection = intersection_dict[i]
    new_int = build_intersection(intersection)

    # new_int["central"][3]["num_incoming"] = 3
    # new_int["central"][3].pop("left_turn_only_length")
    gen_net(new_int["central"], new_int["neighbors"], working_dir=Path("../data/salt-lake-city/"), output=f"sumo/net/{i}.net.xml", open_netedit=False)

0
Parsing nodes from '../data/salt-lake-city/nodes.nod.xml' ... done.
Parsing edges from '../data/salt-lake-city/edges.edg.xml' ... done.
Parsing connections from '../data/salt-lake-city/connections.con.xml' ... done.
 Import done:
   17 nodes loaded.
   32 edges loaded.
Removing self-loops ... done (0ms).
Removing empty nodes ... done (1ms).
   0 nodes removed.
Moving network to origin ... done (0ms).
Computing turning directions ... done (0ms).
Assigning nodes to traffic lights ... done (3ms).
Sorting nodes' edges ... done (0ms).
Computing node shapes ... done (0ms).
Computing edge shapes ... done (1ms).
Computing node types ... done (0ms).
Computing priorities ... done (0ms).
Computing approached edges ... done (0ms).
Guessing and setting roundabouts ... done (0ms).
Computing approaching lanes ... done (0ms).
Dividing of lanes on approached lanes ... done (0ms).
Processing turnarounds ... done (0ms).
Rechecking of lane endings ... done (0ms).
Computing traffic light control informat

Parsing nodes from '../data/salt-lake-city/nodes.nod.xml' ... done.
Parsing edges from '../data/salt-lake-city/edges.edg.xml' ... done.
Parsing connections from '../data/salt-lake-city/connections.con.xml' ... done.
 Import done:
   13 nodes loaded.
   24 edges loaded.
Removing self-loops ... done (0ms).
Removing empty nodes ... done (0ms).
   0 nodes removed.
Moving network to origin ... done (0ms).
Computing turning directions ... done (0ms).
Assigning nodes to traffic lights ... done (0ms).
Sorting nodes' edges ... done (0ms).
Computing node shapes ... done (1ms).
Computing edge shapes ... done (0ms).
Computing node types ... done (0ms).
Computing priorities ... done (0ms).
Computing approached edges ... done (0ms).
Guessing and setting roundabouts ... done (0ms).
Computing approaching lanes ... done (0ms).
Dividing of lanes on approached lanes ... done (0ms).
Processing turnarounds ... done (0ms).
Rechecking of lane endings ... 
done (0ms).
Computing traffic light control informati

Parsing nodes from '../data/salt-lake-city/nodes.nod.xml' ... done.
Parsing edges from '../data/salt-lake-city/edges.edg.xml' ... done.
Parsing connections from '../data/salt-lake-city/connections.con.xml' ... done.
 Import done:
   17 nodes loaded.
   24 edges loaded.
Removing self-loops ... done (0ms).
Removing empty nodes ... done (0ms).
   0 nodes removed.
Moving network to origin ... done (0ms).
Computing turning directions ... done (0ms).
Assigning nodes to traffic lights ... done (0ms).
Sorting nodes' edges ... done (0ms).
Computing node shapes ... done (1ms).
Computing edge shapes ... done (0ms).
Computing node types ... done (0ms).
Computing priorities ... done (0ms).
Computing approached edges ... done (0ms).
Guessing and setting roundabouts ... done (0ms).
Computing approaching lanes ... done (0ms).
Dividing of lanes on approached lanes ... done (0ms).
Processing turnarounds ... done (0ms).
Rechecking of lane endings ... done (0ms).
Computing traffic light control informatio

Parsing nodes from '../data/salt-lake-city/nodes.nod.xml' ... done.
Parsing edges from '../data/salt-lake-city/edges.edg.xml' ... done.
Parsing connections from '../data/salt-lake-city/connections.con.xml' ... done.
 Import done:
   17 nodes loaded.
   32 edges loaded.
Removing self-loops ... done (0ms).
Removing empty nodes ... done (0ms).
   0 nodes removed.
Moving network to origin ... done (0ms).
Computing turning directions ... done (0ms).
Assigning nodes to traffic lights ... done (0ms).
Sorting nodes' edges ... done (0ms).
Computing node shapes ... done (0ms).
Computing edge shapes ... done (1ms).
Computing node types ... done (0ms).
Computing priorities ... done (0ms).
Computing approached edges ... done (0ms).
Guessing and setting roundabouts ... done (0ms).
Computing approaching lanes ... done (0ms).
Dividing of lanes on approached lanes ... done (0ms).
Processing turnarounds ... done (0ms).
Rechecking of lane endings ... done (0ms).
Computing traffic light control informatio

Parsing nodes from '../data/salt-lake-city/nodes.nod.xml' ... done.
Parsing edges from '../data/salt-lake-city/edges.edg.xml' ... done.
Parsing connections from '../data/salt-lake-city/connections.con.xml' ... done.
 Import done:
   17 nodes loaded.
   32 edges loaded.
Removing self-loops ... done (0ms).
Removing empty nodes ... done (0ms).
   0 nodes removed.
Moving network to origin ... done (0ms).
Computing turning directions ... done (0ms).
Assigning nodes to traffic lights ... done (0ms).
Sorting nodes' edges ... done (1ms).
Computing node shapes ... done (0ms).
Computing edge shapes ... done (0ms).
Computing node types ... done (0ms).
Computing priorities ... done (0ms).
Computing approached edges ... done (0ms).
Guessing and setting roundabouts ... done (0ms).
Computing approaching lanes ... done (0ms).
Dividing of lanes on approached lanes ... done (0ms).
Processing turnarounds ... done (0ms).
Rechecking of lane endings ... done (0ms).
Computing traffic light control informatio

...

done (0ms).
Computing node types ... done (0ms).
Computing priorities ... done (0ms).
Computing approached edges ... done (0ms).
Guessing and setting roundabouts ... done (0ms).
Computing approaching lanes ... done (0ms).
Dividing of lanes on approached lanes ... done (0ms).
Processing turnarounds ... done (0ms).
Rechecking of lane endings ... 
done (0ms).
Computing traffic light control information ... done (0ms).
Computing node logics ... done (1ms).
Computing traffic light logics ... 
done (0ms).
 9 traffic light(s) computed.
Building inner edges ... 
done (0ms).
-----------------------------------------------------
Summary:
 Node type statistics:
  Unregulated junctions       : 0
  Dead-end junctions          : 8
  Priority junctions          : 1
  Right-before-left junctions : 0
  Traffic light junctions      : 9
 Network boundaries:
  Original boundary  : -617.14,-1228.13,1507.55,753.78
  Applied offset     : 617.14,1228.13
  Converted boundary : 0.00,0.00,2124.69,1981.90
-------

Parsing nodes from '../data/salt-lake-city/nodes.nod.xml' ... done.
Parsing edges from '../data/salt-lake-city/edges.edg.xml' ... done.
Parsing connections from '../data/salt-lake-city/connections.con.xml' ... done.
 Import done:
   18 nodes loaded.
   33 edges loaded.
Removing self-loops ... done (0ms).
Removing empty nodes ... done (1ms).
   0 nodes removed.
Moving network to origin ... done (0ms).
Computing turning directions ... done (0ms).
Assigning nodes to traffic lights ... done (0ms).
Sorting nodes' edges ... done (0ms).
Computing node shapes ... done (0ms).
Computing edge shapes ... done (0ms).
Computing node types ... done (0ms).
Computing priorities ... done (0ms).
Computing approached edges ... done (0ms).
Guessing and setting roundabouts ... done (0ms).
Computing approaching lanes ... done (0ms).
Dividing of lanes on approached lanes ... done (0ms).
Processing turnarounds ... done (0ms).
Rechecking of lane endings ... done (0ms).
Computing traffic light control informatio

Parsing nodes from '../data/salt-lake-city/nodes.nod.xml' ... done.
Parsing edges from '../data/salt-lake-city/edges.edg.xml' ... done.
Parsing connections from '../data/salt-lake-city/connections.con.xml' ... done.
 Import done:
   17 nodes loaded.
   32 edges loaded.
Removing self-loops ... done (0ms).
Removing empty nodes ... done (0ms).
   0 nodes removed.
Moving network to origin ... done (0ms).
Computing turning directions ... done (0ms).
Assigning nodes to traffic lights ... done (0ms).
Sorting nodes' edges ... done (0ms).
Computing node shapes ... done (0ms).
Computing edge shapes ... done (0ms).
Computing node types ... done (0ms).
Computing priorities ... done (0ms).
Computing approached edges ... done (0ms).
Guessing and setting roundabouts ... done (0ms).
Computing approaching lanes ... done (0ms).
Dividing of lanes on approached lanes ... done (0ms).
Processing turnarounds ... done (0ms).
Rechecking of lane endings ... done (0ms).
Computing traffic light control informatio

Parsing nodes from '../data/salt-lake-city/nodes.nod.xml' ... done.
Parsing edges from '../data/salt-lake-city/edges.edg.xml' ... done.
Parsing connections from '../data/salt-lake-city/connections.con.xml' ... done.
 Import done:
   18 nodes loaded.
   33 edges loaded.
Removing self-loops ... done (0ms).
Removing empty nodes ... done (0ms).
   0 nodes removed.
Moving network to origin ... done (0ms).
Computing turning directions ... done (0ms).
Assigning nodes to traffic lights ... done (0ms).
Sorting nodes' edges ... done (0ms).
Computing node shapes ... done (1ms).
Computing edge shapes ... done (0ms).
Computing node types ... done (0ms).
Computing priorities ... done (0ms).
Computing approached edges ... done (0ms).
Guessing and setting roundabouts ... done (0ms).
Computing approaching lanes ... done (0ms).
Dividing of lanes on approached lanes ... done (0ms).
Processing turnarounds ... done (0ms).
Rechecking of lane endings ... done (0ms).
Computing traffic light control informatio

Parsing nodes from '../data/salt-lake-city/nodes.nod.xml' ... done.
Parsing edges from '../data/salt-lake-city/edges.edg.xml' ... done.
Parsing connections from '../data/salt-lake-city/connections.con.xml' ... done.
 Import done:
   17 nodes loaded.
   32 edges loaded.
Removing self-loops ... done (0ms).
Removing empty nodes ... done (0ms).
   0 nodes removed.
Moving network to origin ... done (0ms).
Computing turning directions ... done (0ms).
Assigning nodes to traffic lights ... done (0ms).
Sorting nodes' edges ... done (0ms).
Computing node shapes ... done (0ms).
Computing edge shapes ... done (0ms).
Computing node types ... done (0ms).
Computing priorities ... done (0ms).
Computing approached edges ... done (0ms).
Guessing and setting roundabouts ... done (0ms).
Computing approaching lanes ... done (0ms).
Dividing of lanes on approached lanes ... done (0ms).
Processing turnarounds ... done (0ms).
Rechecking of lane endings ... done (0ms).
Computing traffic light control informatio

In [76]:
intersection
new_int = build_intersection(intersection)
 
    # new_int["central"][3]["num_incoming"] = 3
    # new_int["central"][3].pop("left_turn_only_length")
gen_net(new_int["central"], new_int["neighbors"], working_dir=Path("../data/salt-lake-city/"), output=f"sumo/net/{i}.net.xml")

{0: {'length': 156.69388352321886,
  'geometry': [[40.7584903, -111.879646],
   [40.7584899, -111.8794375],
   [40.7584881, -111.8786911],
   [40.7584875, -111.8782384]],
  'road_id': 1024519079,
  'theta': 359.85544434317325,
  'theta_long': 359.8501144163669,
  'direction': 'in',
  'lane_data': [4, 0, 0],
  'lane_data_source': 'osm',
  'turn_data': ['left;through', 'through', 'through', 'right'],
  'turn_data_source': 'osm',
  'oneway': 'yes',
  'highway': 'primary',
  'speed': 30,
  'gradient': -0.0115824,
  'left_turn_only': False},
 1: {'length': 320.4278507378273,
  'geometry': [[40.7584903, -111.879646],
   [40.7583306, -111.8796456],
   [40.7579613, -111.8796448],
   [40.7568834, -111.8796421],
   [40.756829, -111.8796422],
   [40.7564716, -111.8796414],
   [40.7563015, -111.879641]],
  'road_id': 1025533354,
  'theta': 270.1091234878354,
  'theta_long': 270.09952538510373,
  'direction': 'in',
  'lane_data': [2, 0, 2],
  'lane_data_source': 'osm',
  'turn_data': ['left;through

In [59]:
intersection[1]["lane_data"] = [1, 0, 1]
intersection[1]["turn_data"] = ["through;right"]
new_int = build_intersection(intersection)
gen_net(new_int["central"], new_int["neighbors"], working_dir=Path("../data/salt-lake-city/"), output=f"sumo/net/{i}.net.xml", open_netedit=True)

Parsing nodes from '../data/salt-lake-city/nodes.nod.xml' ... done.
Parsing edges from '../data/salt-lake-city/edges.edg.xml' ... done.
Parsing connections from '../data/salt-lake-city/connections.con.xml' ... done.
 Import done:
   18 nodes loaded.
   33 edges loaded.
Removing self-loops ... done (0ms).
Removing empty nodes ... done (0ms).
   0 nodes removed.
Moving network to origin ... done (0ms).
Computing turning directions ... done (0ms).
Assigning nodes to traffic lights ... done (0ms).
Sorting nodes' edges ... done (0ms).
Computing node shapes ... done (1ms).
Computing edge shapes ... done (0ms).
Computing node types ... done (0ms).
Computing priorities ... done (0ms).
Computing approached edges ... done (0ms).
Guessing and setting roundabouts ... done (0ms).
Computing approaching lanes ... done (0ms).
Dividing of lanes on approached lanes ... done (0ms).
Processing turnarounds ... done (0ms).
Rechecking of lane endings ... 
done (0ms).
Computing traffic light control informati

Fontconfig warning: ignoring UTF-8: not a valid region tag


PosixPath('../data/salt-lake-city/sumo/net/2.net.xml')

In [10]:
with open("example.pkl", "wb") as f:
    pickle.dump(new_int, f)

In [52]:
turn_stats = {"right": [90.36729010723205, 9.78688759183337],
              "slight_right": [136.359, 14.275363708151186], 
              "through": [179.70755329451188, 11.986190506968194], 
              "slight_left": [224.36030149677322, 15.710788019224378],
              "left": [269.24091111044555, 8.70355104033318]}


def find_turn(turn, theta, turn_data, index, intersection):
    index_pos = sum([turn in turns for turns in turn_data[:index]])
    index_neg = sum([turn in turns for turns in turn_data[index+1:]])
    def get_degree(a, b):
        return a-b if a>b else a-b+360
    mean, std = turn_stats[turn]
    connection = max(intersection.items(), key=lambda x: scipy.stats.distributions.norm.pdf(get_degree(x[1]["theta"], theta), mean, std))[0]
    target_road = intersection[connection]
    if type(target_road["lane_data"])==list:
        target_lane = target_road["lane_data"][2]
    elif target_road["direction"]=="out":
        target_lane = target_road["lane_data"]
    else:
        return None
        
    if "left" in turn:
        index = max(0, target_lane-1-index_neg)
        return (connection, index)
    if "right" in turn:
        index = min(index_pos, target_lane-1)
        return (connection, index)
    else:
        index = min(target_lane-1, index_pos)
        return (connection, index)
    
def get_connectivity(intersection):
    for i, road1 in intersection.items():
        turn_data = road1["turn_data"][::-1]
        theta = road1["theta"]
        connectivity = {}
        for index, turns in enumerate(turn_data):
            pos, neg = index, len(turn_data)-index-1
            connections = [find_turn(turn, theta, turn_data, index, intersection) for turn in turns.split(";")]
            connections = [connection for connection in connections if connection]
            connectivity[index] = connections
        intersection[i]["connections"] = connectivity
    return intersection


new_intersection_dict = dict()
for index, intersection in intersection_dict.items():
    new_intersection = get_connectivity(intersection)
    new_intersection_dict[index] = new_intersection
intersection_dict = new_intersection_dict

In [54]:
new_intersection_dict = dict()
for idx, intersection in intersection_dict.items():
    new_intersection = []
    for index in range(len(intersection)):
        new_road = {}
        road = intersection[index]
        if type(road["lane_data"])==list:
            new_road["num_incoming"] = road["lane_data"][0]+road["lane_data"][1]
            new_road["num_outgoing"] = road["lane_data"][2]
        else:
            if road["direction"]=="in":
                new_road["num_incoming"] = road["lane_data"]
            else:
                new_road["num_outgoing"] = road["lane_data"]
        if road["left_turn_only"]:
            new_road["left_turn_only_length"] = 50
        if "num_incoming" not in new_road:
            new_road["num_incoming"]=0
        if "num_outgoing" not in new_road:
            new_road["num_outgoing"]=0
        new_road["length"] = road["length"]
        new_road["speed"] = road["speed"]
        new_road["connections"] = {index:[(x, y) for x, y in lst] for index, lst in road["connections"].items()}
        new_road["slope"] = road["gradient"]
        new_intersection.append(new_road)
    new_intersection_dict[idx] = new_intersection
intersection_dict = new_intersection_dict

In [60]:
sample = [intersection for index, intersection in intersection_dict.items() if any(["left_turn_only_length" in road for road in intersection])][0]

In [61]:
sample

[{'num_incoming': 2,
  'num_outgoing': 1,
  'left_turn_only_length': 50,
  'length': 163.6029751330113,
  'speed': 25,
  'connections': {0: [(2, 0), (3, 0)], 1: [(1, 0)]},
  'slope': -0.042976799999999996},
 {'num_outgoing': 1,
  'num_incoming': 0,
  'length': 166.88247530596885,
  'speed': 25,
  'connections': {},
  'slope': -0.0027432},
 {'num_incoming': 1,
  'num_outgoing': 1,
  'length': 166.27097381655352,
  'speed': 25,
  'connections': {0: [(0, 0), (1, 0)]},
  'slope': -0.077724},
 {'num_incoming': 1,
  'num_outgoing': 1,
  'length': 168.34981517942833,
  'speed': 25,
  'connections': {0: [(1, 0), (2, 0)]},
  'slope': -0.0621792}]

In [44]:
for index, intersection_sumo in intersection_dict.items():
    print(index)
    gen_net(intersection_sumo, working_dir=Path("../data/salt-lake-city/"), open_netedit=False, output=f"sumo/net/{index}.net.xml")

0
Parsing nodes from '../data/salt-lake-city/nodes.nod.xml' ... done.
Parsing edges from '../data/salt-lake-city/edges.edg.xml' ... done.
Parsing connections from '../data/salt-lake-city/connections.con.xml' ... done.
 Import done:
   5 nodes loaded.
   8 edges loaded.
Removing self-loops ... done (0ms).
Removing empty nodes ... done (0ms).
   0 nodes removed.
Moving network to origin ... done (0ms).
Computing turning directions ... done (0ms).
Assigning nodes to traffic lights ... done (0ms).
Sorting nodes' edges ... done (0ms).
Computing node shapes ... done (0ms).
Computing edge shapes ... done (0ms).
Computing node types ... done (0ms).
Computing priorities ... done (0ms).
Computing approached edges ... done (0ms).
Guessing and setting roundabouts ... done (0ms).
Computing approaching lanes ... done (0ms).
Dividing of lanes on approached lanes ... done (0ms).
Processing turnarounds ... done (0ms).
Rechecking of lane endings ... done (0ms).
Computing traffic light control informatio

Parsing connections from '../data/salt-lake-city/connections.con.xml' ... done.
 Import done:
   5 nodes loaded.
   8 edges loaded.
Removing self-loops ... done (0ms).
Removing empty nodes ... done (0ms).
   0 nodes removed.
Moving network to origin ... done (0ms).
Computing turning directions ... done (0ms).
Assigning nodes to traffic lights ... done (0ms).
Sorting nodes' edges ... done (0ms).
Computing node shapes ... done (1ms).
Computing edge shapes ... done (0ms).
Computing node types ... done (0ms).
Computing priorities ... done (0ms).
Computing approached edges ... done (0ms).
Guessing and setting roundabouts ... done (0ms).
Computing approaching lanes ... done (0ms).
Dividing of lanes on approached lanes ... done (0ms).
Processing turnarounds ... done (0ms).
Rechecking of lane endings ... done (0ms).
Computing traffic light control information ... done (0ms).
Computing node logics ... done (0ms).
Computing traffic light logics ... 
done (0ms).
 1 traffic light(s) computed.
Buil

Parsing nodes from '../data/salt-lake-city/nodes.nod.xml' ... done.
Parsing edges from '../data/salt-lake-city/edges.edg.xml' ... done.
Parsing connections from '../data/salt-lake-city/connections.con.xml' ... done.
 Import done:
   5 nodes loaded.
   8 edges loaded.
Removing self-loops ... done (0ms).
Removing empty nodes ... done (0ms).
   0 nodes removed.
Moving network to origin ... done (0ms).
Computing turning directions ... done (0ms).
Assigning nodes to traffic lights ... done (0ms).
Sorting nodes' edges ... done (0ms).
Computing node shapes ... done (0ms).
Computing edge shapes ... done (0ms).
Computing node types ... done (0ms).
Computing priorities ... done (0ms).
Computing approached edges ... done (0ms).
Guessing and setting roundabouts ... done (0ms).
Computing approaching lanes ... done (0ms).
Dividing of lanes on approached lanes ... done (0ms).
Processing turnarounds ... done (0ms).
Rechecking of lane endings ... done (0ms).
Computing traffic light control information 

Parsing nodes from '../data/salt-lake-city/nodes.nod.xml' ... done.
Parsing edges from '../data/salt-lake-city/edges.edg.xml' ... done.
Parsing connections from '../data/salt-lake-city/connections.con.xml' ... done.
 Import done:
   5 nodes loaded.
   6 edges loaded.
Removing self-loops ... done (0ms).
Removing empty nodes ... done (0ms).
   0 nodes removed.
Moving network to origin ... done (0ms).
Computing turning directions ... done (0ms).
Assigning nodes to traffic lights ... done (0ms).
Sorting nodes' edges ... done (0ms).
Computing node shapes ... done (0ms).
Computing edge shapes ... done (0ms).
Computing node types ... done (0ms).
Computing priorities ... done (0ms).
Computing approached edges ... done (0ms).
Guessing and setting roundabouts ... done (0ms).
Computing approaching lanes ... done (0ms).
Dividing of lanes on approached lanes ... done (0ms).
Processing turnarounds ... done (0ms).
Rechecking of lane endings ... done (0ms).
Computing traffic light control information 

Parsing nodes from '../data/salt-lake-city/nodes.nod.xml' ... done.
Parsing edges from '../data/salt-lake-city/edges.edg.xml' ... done.
Parsing connections from '../data/salt-lake-city/connections.con.xml' ... done.
 Import done:
   4 nodes loaded.
   6 edges loaded.
Removing self-loops ... done (0ms).
Removing empty nodes ... done (0ms).
   0 nodes removed.
Moving network to origin ... done (0ms).
Computing turning directions ... done (0ms).
Assigning nodes to traffic lights ... done (0ms).
Sorting nodes' edges ... done (0ms).
Computing node shapes ... done (0ms).
Computing edge shapes ... done (0ms).
Computing node types ... done (0ms).
Computing priorities ... done (0ms).
Computing approached edges ... done (0ms).
Guessing and setting roundabouts ... done (0ms).
Computing approaching lanes ... done (0ms).
Dividing of lanes on approached lanes ... done (0ms).
Processing turnarounds ... done (0ms).
Rechecking of lane endings ... 
done (0ms).
Computing traffic light control information

...

Parsing nodes from '../data/salt-lake-city/nodes.nod.xml' ... done.
Parsing edges from '../data/salt-lake-city/edges.edg.xml' ... done.
Parsing connections from '../data/salt-lake-city/connections.con.xml' ... done.
 Import done:
   5 nodes loaded.
   8 edges loaded.
Removing self-loops ... done (0ms).
Removing empty nodes ... done (0ms).
   0 nodes removed.
Moving network to origin ... done (0ms).
Computing turning directions ... done (0ms).
Assigning nodes to traffic lights ... done (0ms).
Sorting nodes' edges ... done (0ms).
Computing node shapes ... done (0ms).
Computing edge shapes ... done (0ms).
Computing node types ... done (0ms).
Computing priorities ... done (0ms).
Computing approached edges ... done (0ms).
Guessing and setting roundabouts ... done (0ms).
Computing approaching lanes ... done (0ms).
Dividing of lanes on approached lanes ... done (0ms).
Processing turnarounds ... done (0ms).
Rechecking of lane endings ... done (0ms).
Computing traffic light control information 

Parsing nodes from '../data/salt-lake-city/nodes.nod.xml' ... done.
Parsing edges from '../data/salt-lake-city/edges.edg.xml' ... done.
Parsing connections from '../data/salt-lake-city/connections.con.xml' ... done.
 Import done:
   5 nodes loaded.
   8 edges loaded.
Removing self-loops ... done (0ms).
Removing empty nodes ... done (0ms).
   0 nodes removed.
Moving network to origin ... done (0ms).
Computing turning directions ... done (0ms).
Assigning nodes to traffic lights ... done (0ms).
Sorting nodes' edges ... done (0ms).
Computing node shapes ... done (0ms).
Computing edge shapes ... done (1ms).
Computing node types ... done (0ms).
Computing priorities ... done (0ms).
Computing approached edges ... done (0ms).
Guessing and setting roundabouts ... done (0ms).
Computing approaching lanes ... done (0ms).
Dividing of lanes on approached lanes ... done (0ms).
Processing turnarounds ... done (0ms).
Rechecking of lane endings ... done (0ms).
Computing traffic light control information 

Parsing nodes from '../data/salt-lake-city/nodes.nod.xml' ... done.
Parsing edges from '../data/salt-lake-city/edges.edg.xml' ... done.
Parsing connections from '../data/salt-lake-city/connections.con.xml' ... done.
 Import done:
   4 nodes loaded.
   6 edges loaded.
Removing self-loops ... done (0ms).
Removing empty nodes ... done (0ms).
   0 nodes removed.
Moving network to origin ... done (0ms).
Computing turning directions ... done (0ms).
Assigning nodes to traffic lights ... done (0ms).
Sorting nodes' edges ... done (0ms).
Computing node shapes ... done (0ms).
Computing edge shapes ... done (0ms).
Computing node types ... done (0ms).
Computing priorities ... done (0ms).
Computing approached edges ... done (0ms).
Guessing and setting roundabouts ... done (0ms).
Computing approaching lanes ... done (0ms).
Dividing of lanes on approached lanes ... done (0ms).
Processing turnarounds ... done (0ms).
Rechecking of lane endings ... done (0ms).
Computing traffic light control information 

Parsing nodes from '../data/salt-lake-city/nodes.nod.xml' ... done.
Parsing edges from '../data/salt-lake-city/edges.edg.xml' ... done.
Parsing connections from '../data/salt-lake-city/connections.con.xml' ... done.
 Import done:
   5 nodes loaded.
   8 edges loaded.
Removing self-loops ... done (0ms).
Removing empty nodes ... done (0ms).
   0 nodes removed.
Moving network to origin ... done (0ms).
Computing turning directions ... done (0ms).
Assigning nodes to traffic lights ... done (0ms).
Sorting nodes' edges ... done (0ms).
Computing node shapes ... done (0ms).
Computing edge shapes ... done (0ms).
Computing node types ... done (0ms).
Computing priorities ... done (0ms).
Computing approached edges ... done (0ms).
Guessing and setting roundabouts ... done (0ms).
Computing approaching lanes ... done (0ms).
Dividing of lanes on approached lanes ... done (0ms).
Processing turnarounds ... done (0ms).
Rechecking of lane endings ... done (0ms).
Computing traffic light control information 

Parsing nodes from '../data/salt-lake-city/nodes.nod.xml' ... done.
Parsing edges from '../data/salt-lake-city/edges.edg.xml' ... done.
Parsing connections from '../data/salt-lake-city/connections.con.xml' ... done.
 Import done:
   5 nodes loaded.
   6 edges loaded.
Removing self-loops ... done (0ms).
Removing empty nodes ... done (0ms).
   0 nodes removed.
Moving network to origin ... done (0ms).
Computing turning directions ... done (0ms).
Assigning nodes to traffic lights ... done (0ms).
Sorting nodes' edges ... done (0ms).
Computing node shapes ... done (0ms).
Computing edge shapes ... done (0ms).
Computing node types ... done (0ms).
Computing priorities ... done (0ms).
Computing approached edges ... done (0ms).
Guessing and setting roundabouts ... done (0ms).
Computing approaching lanes ... done (0ms).
Dividing of lanes on approached lanes ... done (0ms).
Processing turnarounds ... done (0ms).
Rechecking of lane endings ... done (0ms).
Computing traffic light control information 